# Estimating average causal effects

This notebook illustrates how to estimate average causal effects with ArCO-GP.

In [ ]:
# imports
%reload_ext autoreload
%autoreload 2
import os

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator
from sklearn.neighbors import KernelDensity

from src.abci_arco_gp import ABCIArCOGP as ABCI
from src.config import ABCIArCOGPConfig
from src.environments.generic_environments import *
from src.utils.causal_orders import *
from src.utils.plotting import init_plot_style

init_plot_style()


First, we generate a ground truth environment/SCM.


In [ ]:
# init environment
num_nodes = 5
env_cfg = EnvironmentConfig()
env_cfg.num_observational_train_samples = 200
env_cfg.normalise_data = True
env_cfg.generate_static_intr_dataset = True
env_cfg.num_test_interventions = 1
env_cfg.num_interventional_test_samples = 100

env = BarabasiAlbert(num_nodes, env_cfg)
# env = SachsGraphGeneric(env_cfg)

# plot true graph
nx.draw(env.graph, nx.circular_layout(env.graph), labels=dict(zip(env.graph.nodes, env.graph.nodes)))


Here, we create an ABCI instance with the desired experimental design policy.

In [ ]:

cfg = ABCIArCOGPConfig()
cfg.policy = 'static-obs-dataset'
cfg.max_ps_size = 2
cfg.num_arco_steps = 100
cfg.num_mc_cos = 50
cfg.num_mc_graphs = 5
cfg.num_samples_per_graph = 5
abci = ABCI(env, cfg)



We can now run a number of ABCI loops.

In [ ]:

abci.run()

# plot loss over experiments
ax = plt.figure().gca()
plt.plot(abci.stats['arco_loss'], label='arco_loss')
plt.xlabel('Number of Steps')
plt.ylabel('Loss')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.legend()

Print structure learning stats.

In [ ]:
print()
print(f"ESHD {abci.stats['eshd']} vs. ESHD CPDAG {abci.stats['eshd_cpdag']}")
print(f"True Num E {env.graph.number_of_edges()} vs. E-NUM Edges{abci.stats['enum_edges']}")
print(f"A-AID {abci.stats['aaid']}   vs. A-AID cpdag {abci.stats['aaid_cpdag']}")
print(f"P-AID {abci.stats['paid']}   vs. P-AID cpdag {abci.stats['paid_cpdag']}")
print(f"OSET-AID {abci.stats['oset_aid']} vs. OSET-AID cpdag {abci.stats['oset_aid_cpdag']} ")
print(f"ORDER-AID {abci.stats['order_aid']} ")
print()
print(f"F1 {abci.stats['ef1']}     vs. F1 cpdag {abci.stats['ef1_cpdag']}")
print(f"TPR {abci.stats['etpr']} vs. TPR cpdag {abci.stats['etpr_cpdag']}")
print(f"TNR {abci.stats['etnr']} vs. TNR cpdag {abci.stats['etnr_cpdag']}")
print(f"FNR {abci.stats['efnr']} vs. FNR cpdag {abci.stats['efnr_cpdag']}")
print(f"FPR {abci.stats['efpr']} vs. FPR cpdag {abci.stats['efpr_cpdag']}")
print(f"AUROC {abci.stats['auroc']}  vs. AUROC CPDAG {abci.stats['auroc_cpdag']}")
print(f"AUPRC {abci.stats['auprc']}  vs. AUPRC CPDAG {abci.stats['auprc_cpdag']}")
print()
print(f"MMD {abci.stats['mmd']}, L1 {abci.stats['dmae']}, L2 {abci.stats['dmse']}")


# Sample from posterior interventional distribution.

In [ ]:
# sampling from the true and predicted interventional distributions
target = 'X4'
interventions = {'X1': torch.tensor(1.)}
num_env_samples = 10000
num_samples_per_graph = 10
with torch.no_grad():
    # sample from true interventional distribution
    env_samples = env.sample(interventions, num_env_samples, 1).data[target].squeeze()
    env_weights = torch.ones_like(env_samples) / env_samples.numel()
    env_mean = env_samples.mean()

    # sample from learned posterior distribution
    samples, weights = abci.sample(interventions, num_samples_per_graph)
    samples = samples[target]
    posterior_mean = weights @ samples

print(f'E[{target} | do({interventions})] = {env_mean} (TRUE) vs. {posterior_mean} (PREDICTION)')


Plot the empirical posterior interventional distribution.

In [ ]:

# plotting params
xrange = (-3., 3.)
num_bins = 200
bin_width = (xrange[1] - xrange[0]) / num_bins
true_color = '#5DADE2'
pred_color = '#911225'
alpha = 0.3
kernel_bw = 0.2
lw = 7

min_x = min(env_samples.min(), samples.min()).item()
max_x = max(env_samples.max(), samples.max()).item()
support = torch.linspace(min_x, max_x, steps=500).unsqueeze(-1)

xrange = (min_x, max_x)

plt.figure(figsize=(12, 7))

# true distribution
hist, bin_edges = torch.histogram(env_samples, bins=num_bins, range=xrange, weight=env_weights, density=True)
plt.bar(bin_edges[:-1], hist, align='edge', width=bin_width, color=true_color, alpha=alpha)

kde = KernelDensity(kernel="gaussian", bandwidth=kernel_bw)
kde.fit(env_samples.view(-1, 1), sample_weight=env_weights)
log_dens = kde.score_samples(support)
plt.plot(support, np.exp(log_dens), '--', color=true_color, lw=lw)

weights = weights + 1e-10
# predicted distribution
hist, bin_edges = torch.histogram(samples, bins=num_bins, range=xrange, weight=weights, density=True)
plt.bar(bin_edges[:-1], hist, align='edge', width=bin_width, color=pred_color, alpha=alpha)

kde = KernelDensity(kernel="gaussian", bandwidth=kernel_bw)
kde.fit(samples.view(-1, 1), sample_weight=weights)
log_dens = kde.score_samples(support)
plt.plot(support, np.exp(log_dens), color=pred_color, lw=lw)

# means
eps = 1e-2
plt.plot([env_mean - eps, env_mean - eps], [0., max(hist)], '--', color='black', lw=lw)
plt.plot([env_mean + eps, env_mean + eps], [0., max(hist)], '--', color='black', lw=lw)
plt.plot([env_mean, env_mean], [0., max(hist)], '--', color=true_color, label='True ACE', lw=lw)

plt.plot([posterior_mean - eps, posterior_mean - eps], [0., max(hist)], color='black', lw=lw)
plt.plot([posterior_mean + eps, posterior_mean + eps], [0., max(hist)], color='black', lw=lw)
plt.plot([posterior_mean, posterior_mean], [0., max(hist)], color=pred_color, label='Predicted ACE', lw=lw)

# label
intr_str = ', '.join([f'do({k}={v.int()})' for k, v in interventions.items()])
plt.xlabel(f'{target} | {intr_str}')
# plt.xlim([-2.5, 2.5])
plt.tight_layout()

save_plots = False
dpi = 400
fig_format = 'pdf'
fig_dir = '../figures/'
figdate = '20241011'
fig_name = f'{env.__class__.__name__}-{target}-do({",".join(interventions.keys())})'

os.makedirs(fig_dir, exist_ok=True)

if save_plots:
    plt.savefig(fig_dir + f'{figdate}-{fig_name}.{fig_format}', dpi=dpi, bbox_inches='tight')
